In [1]:
! python -V

Python 3.9.7


In [6]:
! pip install transformers

     |████████████████████████████████| 3.1 MB 626 kB/s 
     |████████████████████████████████| 661 kB 9.8 MB/s 
     |████████████████████████████████| 762 kB 9.7 MB/s 
     |████████████████████████████████| 59 kB 4.7 MB/s 
     |████████████████████████████████| 62 kB 712 kB/s 
     |████████████████████████████████| 76 kB 3.7 MB/s 
     |████████████████████████████████| 3.3 MB 9.8 MB/s 
     |████████████████████████████████| 40 kB 4.5 MB/s 
     |████████████████████████████████| 895 kB 10.4 MB/s 
     |████████████████████████████████| 67 kB 4.4 MB/s 
     |████████████████████████████████| 138 kB 10.4 MB/s 
     |████████████████████████████████| 61 kB 5.1 MB/s 
     |████████████████████████████████| 97 kB 4.7 MB/s 
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.0.4
    Uninstalling pyparsing-3.0.4:
      Successfully uninstalled pyparsing-3.0.4


In [1]:
import random 
import time
import copy

import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.cuda.amp import autocast, GradScaler

from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from transformers import BertTokenizer, BertModel, AdamW
from transformers import RobertaTokenizer, RobertaModel

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from torch.utils.data import Dataset, DataLoader
from word_aug import get_aug_dataset


[nltk_data] Downloading package punkt to /home/andrew/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
torch.cuda.is_available()

True

In [3]:
#CHECKPOINT = "sberbank-ai/sbert_large_nlu_ru"
MAX_LEN = 256
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
EPOCHS = 3
LEARNING_RATE = 2e-05
#tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)
CHECKPOINT = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(CHECKPOINT)

def get_individual_labels(df):
    labels = pd.get_dummies(df.label).rename({
        "ДЖОУИ": "Joey", "МОНИКА": "Monica", "РЕЙЧЕЛ": "Rachel", "РОСС": "Ross", 
        "ФИБИ": "Phoebe", "ЧЕНДЛЕР": "Chandler"
    }, axis=1)
    return pd.concat([df, labels], axis=1)

In [4]:
%cd friends-classification/
!mkdir models
! ls

/home/andrew/ml/friends-classification
mkdir: cannot create directory ‘models’: File exists
english
example_eng_ru.csv
fb_model_translate_en_ru_2_friend_response_test.csv
fb_model_translate_en_ru_2_friend_response_train.csv
fb_model_translate_en_ru_2_friend_response_val.csv
fb_model_translate_en_ru_2_other_speaker_test.csv
fb_model_translate_en_ru_2_other_speaker_train.csv
fb_model_translate_en_ru_2_other_speaker_val.csv
fb_model_translate_ru_en_1_friend_response_test.csv
fb_model_translate_ru_en_1_friend_response_train.csv
fb_model_translate_ru_en_1_friend_response_val.csv
fb_model_translate_ru_en_1_other_speaker_test.csv
fb_model_translate_ru_en_1_other_speaker_train.csv
fb_model_translate_ru_en_1_other_speaker_val.csv
final_model.pt
final_submission.csv
helsinki_model_translate_ru_en_1_friend_response_test.csv
helsinki_model_translate_ru_en_1_friend_response_train.csv
helsinki_model_translate_ru_en_1_friend_response_val.csv
helsinki_model_translate_ru_en_1_other_speaker_test.csv
hel

In [5]:
df_train = pd.read_csv('english/df_train_eng.csv').rename({'Category': 'label'}, axis=1)
df_train.other_speaker.fillna('', inplace=True)
df_train.friend_response.fillna('', inplace=True)
df_val = pd.read_csv('english/df_val_eng.csv')
df_val.other_speaker.fillna('', inplace=True)
df_val.friend_response.fillna('', inplace=True)
df_test = pd.read_csv('english/df_test_eng.csv')
df_test.other_speaker.fillna('', inplace=True)
df_test.friend_response.fillna('', inplace=True)

df_train = get_individual_labels(df_train)
df_val = get_individual_labels(df_val)

# Encoding target variable
names_to_cats = LabelEncoder()
df_train['label_code'] = names_to_cats.fit_transform(df_train.label)
df_val['label_code'] = names_to_cats.transform(df_val.label)
df_fb_train = pd.read_csv('train_data_rus_fb_model.csv')
df_full = pd.concat([df_train, df_val])
print(df_train["label"].value_counts()/df_train.shape[0])
print()
print(df_val["label"].value_counts()/df_val.shape[0])

РОСС       0.176569
РЕЙЧЕЛ     0.176089
ЧЕНДЛЕР    0.170568
ДЖОУИ      0.166287
МОНИКА     0.160525
ФИБИ       0.149962
Name: label, dtype: float64

РОСС       0.176746
РЕЙЧЕЛ     0.176026
ЧЕНДЛЕР    0.170626
ДЖОУИ      0.166307
МОНИКА     0.160547
ФИБИ       0.149748
Name: label, dtype: float64


In [6]:
class FriendsDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512, padding='max_length', 
                 with_labels=True):

        self.dataframe = dataframe  # pandas dataframe
        #Initialize the tokenizer
        self.tokenizer = tokenizer  
        self.padding = padding
        self.max_length = max_length
        
        self.with_labels = with_labels 
        if 'label' not in self.dataframe.columns:
          self.with_labels = False

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):

        # Selecting sentence1 and sentence2 at the specified index in the data frame
        sent1 = self.dataframe.other_speaker.iloc[index]
        sent2 = self.dataframe.friend_response.iloc[index]

        # Tokenize the pair of sentences to get token ids, attention masks and token type ids
        encoded_pair = self.tokenizer(sent1, sent2, 
                                      padding=self.padding,  # Pad to max_length
                                      truncation=True,  # Truncate to max_length
                                      max_length=self.max_length,  
                                      return_tensors='pt')  # Return torch.Tensor objects
        
        token_ids = encoded_pair['input_ids'].squeeze(0)  # tensor of token ids
        attn_masks = encoded_pair['attention_mask'].squeeze(0)  # binary tensor with "0" for padded values and "1" for the other values
        token_type_ids = encoded_pair['token_type_ids'].squeeze(0)  # binary tensor with "0" for the 1st sentence tokens & "1" for the 2nd sentence tokens

        if self.with_labels:  # True if the dataset has labels
            label = self.dataframe.label_code.iloc[index]
            #label = self.dataframe.Phoebe.iloc[index]
            return token_ids, attn_masks, token_type_ids, label  
        else:
            return token_ids, attn_masks, token_type_ids

In [7]:
class SentencePairClassifier(nn.Module):

    def __init__(self, model=CHECKPOINT, freeze_model=True):
        super(SentencePairClassifier, self).__init__()
        #  Instantiating BERT-based model object
        # self.pretrained_layer = AutoModel.from_pretrained(CHECKPOINT)
        self.pretrained_layer = BertModel.from_pretrained(CHECKPOINT)
        #self.pretrained_layer = RobertaModel.from_pretrained(CHECKPOINT)

        hidden_size = self.pretrained_layer.config.hidden_size

        # Freeze model layers and only train the classification layer weights
        if freeze_model:
            for p in self.pretrained_layer.parameters():
                p.requires_grad = False
            print('All parameters frozen')
        # Classification layer
        self.cls_layer = nn.Linear(hidden_size, 6)

        self.dropout = nn.Dropout(p=0.3)

    @autocast()  # run in mixed precision
    def forward(self, input_ids, attn_masks, token_type_ids):
        '''
        Inputs:
            -input_ids : Tensor  containing token ids
            -attn_masks : Tensor containing attention masks to be used to focus on non-padded values
            -token_type_ids : Tensor containing token type ids to be used to identify sentence1 and sentence2
        '''

        # Feeding the inputs to the BERT-based model to obtain contextualized representations
        output = self.pretrained_layer(input_ids, attn_masks, token_type_ids)

        # Feeding to the classifier layer the last layer hidden-state of the [CLS] token further processed by a
        # Linear Layer and a Tanh activation. The Linear layer weights were trained from the sentence order prediction (ALBERT) or next sentence prediction (BERT)
        # objective during pre-training.
        logits = self.cls_layer(self.dropout(output.pooler_output))

        return logits

In [8]:
def set_seed(seed):
    """ Set all seeds to make results reproducible """
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    #os.environ['PYTHONHASHSEED'] = str(seed)
    
@autocast()
def evaluate_loss(net, device, criterion, dataloader):
    net.eval()
    n_correct = 0
    mean_loss = 0
    count = 0

    with torch.no_grad():
        for it, (seq, attn_masks, token_type_ids, labels) in enumerate(tqdm(dataloader)):
            seq, attn_masks, token_type_ids, labels = \
                seq.to(device), attn_masks.to(device), token_type_ids.to(device), labels.to(device)
            logits = net(seq, attn_masks, token_type_ids)
            mean_loss += criterion(logits.squeeze(-1), labels).item()
            count += 1
            max_logits, argmax_idx = torch.max(logits.data, dim=1)
            n_correct += calcuate_accu(argmax_idx, labels)
    del logits
    return mean_loss / count, n_correct / len(dataloader.dataset)
  
# Function to calcuate the accuracy of the model
def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

In [9]:
def train_bert(net, criterion, opti, lr, lr_scheduler, train_loader, val_loader, epochs, iters_to_accumulate):

    best_loss = np.Inf
    best_acc = 0
    best_ep = 1
    n_iterations = len(train_loader)
    batch_size = train_loader.batch_size
    print_every = 1000 // batch_size  # print the training loss this many times per epoch
    print_eval_iters = 10000 // batch_size
    scaler = GradScaler()

    for ep in range(epochs):
        net.train()
        curr_loss = 0.0
        curr_n_correct = 0.
        trailing_loss = 0.
        trailing_n_correct = 0.
        curr_n_tr_examples = 0
        trainling_n_tr_examples = 0

        for it, (seq, attn_masks, token_type_ids, labels) in enumerate(tqdm(train_loader)):
            # Converting to cuda tensors
            seq, attn_masks, token_type_ids, labels = \
                seq.to(device), attn_masks.to(device), token_type_ids.to(device), labels.to(device)
  
            # Enables autocasting for the forward pass (model + loss)
            with autocast():
                # Obtaining the logits from the model
                pooled = net(seq, attn_masks, token_type_ids)

                # Computing loss
                loss = criterion(pooled.squeeze(-1), labels)
                #print(loss, type(loss))
                loss = loss / iters_to_accumulate  # Normalize the loss because it is averaged
                # Computing accuracy
                #print(pooled.squeeze(-1), labels)
                curr_loss += loss.item() 
                big_val, big_idx = torch.max(pooled.data, dim=1)
                n_correct = calcuate_accu(big_idx, labels)
                curr_n_correct += n_correct

            trailing_loss += loss.item() 
            trailing_n_correct += n_correct
            curr_n_tr_examples += labels.size(0)
            trainling_n_tr_examples += labels.size(0)

            # Backpropagating the gradients
            # Scales loss.  Calls backward() on scaled loss to create scaled gradients.
            scaler.scale(loss).backward()

            if (it + 1) % iters_to_accumulate == 0:
                # Optimization step
                # scaler.step() first unscales the gradients of the optimizer's assigned params.
                # If these gradients do not contain infs or NaNs, opti.step() is then called,
                # otherwise, opti.step() is skipped.
                scaler.step(opti)
                # Updates the scale for next iteration.
                scaler.update()
                # Adjust the learning rate based on the number of iterations.
                lr_scheduler.step()
                # Clear gradients
                opti.zero_grad()

            if (it + 1) % print_every == 0:  # Print training loss information
                print()
                print("Batch {}/{} of epoch {} complete. Loss per last {} samples:: {} "
                      .format(it+1, n_iterations, ep+1, curr_n_tr_examples, curr_loss / print_every))
                accu_step = (curr_n_correct*100) / curr_n_tr_examples 
                #print(f"Training Loss per 5000 steps: {loss_step}")
                print(f"Training Accuracy per last {curr_n_tr_examples} samples: {accu_step}")
                curr_loss = 0.0
                curr_n_tr_examples = 0
                curr_n_correct = 0


            if (it + 1) % print_eval_iters == 0 or it ==  n_iterations - 1:
                del pooled, loss
                print("Epoch {}, batch {} complete! Training Loss : {}"
                .format(ep+1, it+1, trailing_loss / (it+1)))
                print("Epoch {}, batch {} complete! Training Accuracy : {}"
                .format(ep+1, it+1, trailing_n_correct / trainling_n_tr_examples))
                with autocast():
                    val_loss, val_accuracy = evaluate_loss(net, device, criterion, val_loader)  # Compute validation loss
                #print()
                print("Epoch {}, batch {} complete! Validation Loss : {}".format(ep+1, it+1, val_loss))
                print("Epoch {}, batch {} complete! Validation Accuracy : {}".format(ep+1, it+1,val_accuracy))
                net.train()
                #if val_loss < best_loss:
                if val_accuracy > best_acc:
                    print("Validation loss changed from {} to {}".format(best_loss, val_loss))
                    print("Best validation accuracy improved from {} to {}".format(best_acc, val_accuracy))
                    print()
                    #net_copy = copy.deepcopy(net)  # save a copy of the model
                    best_loss = val_loss
                    best_acc = val_accuracy
                    best_ep = ep + 1
                    # Saving the model
                    path_to_model='models/{}_lr_{}_val_acc_{}_ep_{}.pt'.format(time.ctime(), lr, round(best_acc, 4), best_ep)
                    torch.save(net.state_dict(), path_to_model)
                    print("The model has been saved in {}".format(path_to_model))

    torch.cuda.empty_cache()

In [10]:
from transformers import get_linear_schedule_with_warmup
from transformers import get_constant_schedule
from sklearn.utils import compute_class_weight
#  Set all seeds to make reproducible results
set_seed(1)

# Creating instances of training and validation set
print("Reading training data...")
#train_set = FriendsDataset(dataframe=df_train, tokenizer=tokenizer, max_length=MAX_LEN)
train_set = FriendsDataset(dataframe=df_full, tokenizer=tokenizer, max_length=MAX_LEN)

print("Reading validation data...")
val_set = FriendsDataset(dataframe=df_val, tokenizer=tokenizer, max_length=MAX_LEN)
# Creating instances of training and validation dataloaders
train_loader = DataLoader(train_set, batch_size=TRAIN_BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_set, batch_size=VALID_BATCH_SIZE, shuffle=False, num_workers=2)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = SentencePairClassifier(model=CHECKPOINT, freeze_model=False)
print(device)

if torch.cuda.device_count() > 1:  # if multiple GPUs
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    net = nn.DataParallel(net)

net.to(device)

"""class_weights = compute_class_weight(
    'balanced', 
    classes=np.unique(df_train.Phoebe), y=df_train.Phoebe)
class_weights = torch.tensor(class_weights, dtype=torch.float)
class_weights = class_weights.to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)"""
criterion = nn.CrossEntropyLoss()

opti = AdamW(net.parameters(), lr=LEARNING_RATE, weight_decay=1e-2)
num_warmup_steps = 0 # The number of steps for the warmup phase.
iters_to_accumulate = 2
num_training_steps = EPOCHS * len(train_loader)  # The total number of training steps
t_total = (len(train_loader) // iters_to_accumulate) * EPOCHS  # Necessary to take into account Gradient accumulation
#lr_scheduler = get_linear_schedule_with_warmup(optimizer=opti, num_warmup_steps=num_warmup_steps, num_training_steps=t_total)
lr_scheduler = get_constant_schedule(optimizer=opti)


Reading training data...
Reading validation data...


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


cuda:0


In [11]:
# Training on all availible data
train_bert(net, criterion, opti, LEARNING_RATE, lr_scheduler, train_loader, val_loader, EPOCHS, iters_to_accumulate)

  4%|▎         | 62/1736 [00:43<19:23,  1.44it/s]


Batch 62/1736 of epoch 1 complete. Loss per last 992 samples:: 0.9178887644121724 
Training Accuracy per last 992 samples: 17.842741935483872


  7%|▋         | 124/1736 [01:26<18:56,  1.42it/s]


Batch 124/1736 of epoch 1 complete. Loss per last 992 samples:: 0.9073033486643145 
Training Accuracy per last 992 samples: 16.129032258064516


 11%|█         | 186/1736 [02:09<18:00,  1.43it/s]

...


Batch 1736/1736 of epoch 3 complete. Loss per last 987 samples:: 0.667423673214451 
Training Accuracy per last 987 samples: 48.02431610942249
Epoch 3, batch 1736 complete! Training Loss : 0.6761971951767047
Epoch 3, batch 1736 complete! Training Accuracy : 0.4713550106225919


100%|██████████| 174/174 [00:38<00:00,  4.54it/s]


Epoch 3, batch 1736 complete! Validation Loss : 0.9934600447786266
Epoch 3, batch 1736 complete! Validation Accuracy : 0.6587473002159827
Validation loss changed from 1.0959558904856102 to 0.9934600447786266
Best validation accuracy improved from 0.6069114470842333 to 0.6587473002159827



100%|██████████| 1736/1736 [20:18<00:00,  1.43it/s]

The model has been saved in models/Thu Nov 25 02:35:15 2021_lr_2e-05_val_acc_0.6587_ep_3.pt


In [12]:
torch.save(net.state_dict(), 'models/orig_final_model.pt')

In [16]:
#net.load_state_dict(torch.load('models/orig_final_model.pt'))

<All keys matched successfully>

In [14]:
test_set = FriendsDataset(dataframe=df_test, tokenizer=tokenizer, max_length=MAX_LEN)
test_loader = DataLoader(test_set, batch_size=VALID_BATCH_SIZE, shuffle=False, num_workers=1)
def predict(net, device, dataloader):
    net.eval()
    predictions = []

    with torch.no_grad():
        for it, (seq, attn_masks, token_type_ids) in enumerate(tqdm(dataloader)):
            seq, attn_masks, token_type_ids = \
                seq.to(device), attn_masks.to(device), token_type_ids.to(device)
            logits = net(seq, attn_masks, token_type_ids)
            max_logits, argmax_idx = torch.max(logits.data, dim=1)
            predictions.extend(argmax_idx.tolist())
    del logits
    return predictions
preds = predict(net, device, test_loader)

100%|██████████| 193/193 [00:46<00:00,  4.18it/s]


In [15]:
df_test

,Unnamed: 0,Id,other_speaker,friend_response
0,0,0,"But I'm unemployed, my music is all I really h...","All right, I'm gonna do it! I'm gonna get shot..."
1,1,1,Check this out. Five hundred and seventeen boxes!,"Oh my God, how did you do that?"
2,2,2,"Okay. Okay. Would, would it help if I went ove...",Yeah! Yeah! That would be very helpful! Yeah.
3,3,3,"Ross, what is taking you so long?","I'm sorry, it's almost as if this wasn't built..."
4,4,4,So who are you?,"Well, our names really are Monica and Chandler..."
...,...,...,...,...
3081,3081,3081,It's not just the drum noise. Every five minut...,"Yes, thank you. You see, this is how normal pe..."
3082,3082,3082,I think I accidentally used Monica’s boxes to ...,Oh no. Dad! Dad! What. Oh God everything’s rui...
3083,3083,3083,"so y'know, that’s why, within a few years, tha...","Oh, this is so great."
3084,3084,3084,He slept with you and then never called you.,And I just wanted a new daddy for Davy and Becky.


In [35]:
df_test

,Id,other_speaker,friend_response
0,0,"Но я безработный, моя музыка - это все, что у ...",Меня застрелят. Любой совет?
1,1,Посмотри. Пятьсот семнадцать коробок!,"Боже мой, как ты это сделал?"
2,2,"Хорошо. Хорошо. Помогло бы, если бы я подошел ...",Это было бы очень полезно!
3,3,"Росс, чего ты так долго?","Простите, это как будто не для быстрого отдыха!"
4,4,Так кто ты?,"Ну, наши имена действительно Моника и Чендлер...."
...,...,...,...
3081,3081,Дело не только в барабанах. Каждые пять минут ...,"Понимаете, именно так нормальные люди должны р..."
3082,3082,"Кажется, я случайно использовал коробки Моники...","Боже, все испорчено! Папа, она будет раздавлена!"
3083,3083,"ну знаете, вот почему через несколько лет расп...","Ой, это так здорово."
3084,3084,"Он переспал с тобой, а потом никогда тебе не з...",А я просто хотела нового папу для Дэви и Бекки.


In [16]:
answers = pd.DataFrame(
    names_to_cats.inverse_transform(preds), 
    index=df_test.Id, columns=["Category"])
answers.to_csv('submission1_orig.csv')
answers

,Category
Id,
0,ДЖОУИ
1,МОНИКА
2,РЕЙЧЕЛ
3,РОСС
4,МОНИКА
...,...
3081,ЧЕНДЛЕР
3082,РОСС
3083,РЕЙЧЕЛ


In [38]:
answers = pd.DataFrame(
    names_to_cats.inverse_transform(preds), 
    index=df_test.Id, columns=["Category"])
answers.to_csv('submission1_orig.csv')
answers

,Category
Id,
0,ФИБИ
1,МОНИКА
2,РЕЙЧЕЛ
3,РОСС
4,ЧЕНДЛЕР
...,...
3081,ФИБИ
3082,МОНИКА
3083,РЕЙЧЕЛ


In [ ]:
from transformers import get_constant_schedule
opti = AdamW(net.parameters(), lr=LEARNING_RATE, weight_decay=1e-2)
lr_scheduler = get_constant_schedule(optimizer=opti)
train_set = FriendsDataset(dataframe=df_train.iloc[5000:], tokenizer=tokenizer, max_length=MAX_LEN)

val_set = FriendsDataset(dataframe=df_val, tokenizer=tokenizer, max_length=MAX_LEN)
# Creating instances of training and validation dataloaders
train_loader = DataLoader(train_set, batch_size=TRAIN_BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_set, batch_size=VALID_BATCH_SIZE, shuffle=False, num_workers=2)

train_bert(net, criterion, opti, LEARNING_RATE, lr_scheduler, train_loader, val_loader, EPOCHS, iters_to_accumulate)

In [22]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [22]:
torch.cuda.get_device_properties(0).total_memory / 1e6

8506.769408

In [12]:
# Training perfomance during usual train-eval on hold-out loop
train_bert(net, criterion, opti, LEARNING_RATE, lr_scheduler, train_loader, val_loader, EPOCHS, iters_to_accumulate)

  4%|▍         | 62/1563 [00:43<17:32,  1.43it/s]


Batch 62/1563 of epoch 1 complete. Loss per last 992 samples:: 0.913238279281124 
Training Accuracy per last 992 samples: 19.556451612903224


  8%|▊         | 124/1563 [01:25<16:41,  1.44it/s]


Batch 124/1563 of epoch 1 complete. Loss per last 992 samples:: 0.9004580590032762 
Training Accuracy per last 992 samples: 17.641129032258064


 12%|█▏        | 186/1563 [02:09<16:10,  1.42it/s]

...


Batch 1550/1563 of epoch 5 complete. Loss per last 992 samples:: 0.35248026636339 
Training Accuracy per last 992 samples: 73.99193548387096


100%|█████████▉| 1562/1563 [17:49<00:00,  1.57it/s]

Epoch 5, batch 1563 complete! Training Loss : 0.35191390614286877
Epoch 5, batch 1563 complete! Training Accuracy : 0.7455287480494538


100%|██████████| 1563/1563 [18:28<00:00,  1.41it/s]

Epoch 5, batch 1563 complete! Validation Loss : 2.06698082712875
Epoch 5, batch 1563 complete! Validation Accuracy : 0.3675305975521958


In [ ]:
# Check that we are using 100% of GPU memory footprint support libraries/code
# from https://github.com/patrickvonplaten/notebooks/blob/master/PyTorch_Reformer.ipynb
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip -q install gputil
!pip -q install psutil
!pip -q install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()
 #!kill -9 -1

Gen RAM Free: 11.1 GB  | Proc size: 5.1 GB
GPU RAM Free: 10137MB | Used: 1304MB | Util  11% | Total 11441MB


In [ ]:
printm()


Gen RAM Free: 11.1 GB  | Proc size: 5.1 GB
GPU RAM Free: 10137MB | Used: 1304MB | Util  11% | Total 11441MB
